<a href="https://colab.research.google.com/github/cedro3/others/blob/master/PIFuHD_movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## セットアップ

In [ ]:
# ライブラリー取得
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install pytorch3d

# githubからpifuhdのコードをコピー
!git clone https://github.com/facebookresearch/pifuhd

# githubからpose-estimationのコードをコピーし、学習済み重みをダウンロード
!git clone https://github.com/Daniil-Osokin/lightweight-human-pose-estimation.pytorch.git
%cd /content/lightweight-human-pose-estimation.pytorch/
!wget https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth

# pifuhdの学習済み重みをダンロード
%cd /content/pifuhd/
!sh ./scripts/download_trained_model.sh

# サンプルビデオダウンロード
import gdown
gdown.download('https://drive.google.com/uc?id=1rrccXA-k-45cUx1MDoLH6O3M4RF5pA_E', 'movie.zip', quiet=False)
! unzip movie.zip

## サンプルビデオから静止画を切り出す
video_file = : ビデオ指定\
interval = ：　静止画の切り出し間隔

In [ ]:
import os
import shutil
import cv2
 
# imagesフォルダーリセット
if os.path.isdir('images'):
    shutil.rmtree('images') 
os.makedirs('images', exist_ok=True)
 
def video_2_images(video_file= './movie/01.mp4', 
                   image_dir='./images/', 
                   image_file='%s.png'):
 
    # Initial setting
    i = 0
    interval = 3
    length = 300 # リミッター
    
    cap = cv2.VideoCapture(video_file)
    while(cap.isOpened()):
        flag, frame = cap.read()  
        if flag == False:  
                break
        if i == length*interval:
                break
        if i % interval == 0:    
           cv2.imwrite(image_dir+image_file % str(int(i/interval)).zfill(6), frame)
        i += 1 
    cap.release()  
 
video_2_images()
list_d = os.listdir('./images')
num = len(list_d)
print(num)

## クロップ

In [ ]:
%cd /content/lightweight-human-pose-estimation.pytorch/
import torch
import cv2
import numpy as np
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
import demo

def get_rect(net, images, height_size):
    net = net.eval()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33
    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs) #, demo=True)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []

        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
            valid_keypoints = np.array(valid_keypoints)
            
            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
              pmin = valid_keypoints.min(0)
              pmax = valid_keypoints.max(0)

              center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int)
              radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
              # if leg is missing, use pelvis to get cropping
              center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int)
              radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
              center[1] += int(0.05*radius)
            else:
              center = np.array([img.shape[1]//2,img.shape[0]//2])
              radius = max(img.shape[1]//2,img.shape[0]//2)

            x1 = center[0] - radius
            y1 = center[1] - radius

            rects.append([x1, y1, 2*radius, 2*radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')

net = PoseEstimationWithMobileNet()
checkpoint = torch.load('checkpoint_iter_370000.pth', map_location='cpu')
load_state(net, checkpoint)

# get_rect ループ
from tqdm import trange
for i in trange(num):
    image_path = '/content/pifuhd/images/'+str(i).zfill(6)+'.png'
    get_rect(net.cuda(), [image_path], 512)

# 全ての rect.txt を同じ値に揃える
import shutil
for i in range(1,num):
    shutil.copyfile('/content/pifuhd/images/000000_rect.txt', '/content/pifuhd/images/'+str(i).zfill(6)+'_rect.txt')

## objファイルの作成

In [ ]:
import os
import shutil

# results/pifuhd_final/reconフォルダーリセット
if os.path.isdir('results/pifuhd_final/recon'):
    shutil.rmtree('results/pifuhd_final/recon') 
os.makedirs('results/pifuhd_final/recon', exist_ok=True)

# objファイル作成
%cd /content/pifuhd/
!python -m apps.simple_test -r 256 --use_rect -i './images'


## objファイルから３Dモデル画像生成

**関数定義**

In [ ]:
import io
import os
import torch
from skimage.io import imread
import numpy as np
import cv2
from tqdm import tqdm_notebook as tqdm
import base64
from IPython.display import HTML

# Util function for loading meshes
from pytorch3d.io import load_objs_as_meshes

from IPython.display import HTML
from base64 import b64encode

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.renderer import (
    look_at_view_transform,
    OpenGLOrthographicCameras, 
    PointLights, 
    DirectionalLights, 
    Materials, 
    RasterizationSettings, 
    MeshRenderer, 
    MeshRasterizer,  
    HardPhongShader,
    TexturesVertex
)

def set_renderer():
    # Setup
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)

    # Initialize an OpenGL perspective camera.
    R, T = look_at_view_transform(2.0, 0, 180) 
    cameras = OpenGLOrthographicCameras(device=device, R=R, T=T)

    raster_settings = RasterizationSettings(
        image_size=512, 
        blur_radius=0.0, 
        faces_per_pixel=1, 
        bin_size = None, 
        max_faces_per_bin = None
    )

    lights = PointLights(device=device, location=((2.0, 2.0, 2.0),))

    renderer = MeshRenderer(
        rasterizer=MeshRasterizer(
            cameras=cameras, 
            raster_settings=raster_settings
        ),
        shader=HardPhongShader(
            device=device, 
            cameras=cameras,
            lights=lights
        )
    )
    return renderer

def get_verts_rgb_colors(obj_path):
  rgb_colors = []

  f = open(obj_path)
  lines = f.readlines()
  for line in lines:
    ls = line.split(' ')
    if len(ls) == 7:
      rgb_colors.append(ls[-3:])

  return np.array(rgb_colors, dtype='float32')[None, :, :]

def generate_image_from_obj(obj_path, image_path, renderer):  
    input_image = cv2.imread(image_path)
    input_image = input_image[:,:input_image.shape[1]//3]
    input_image = cv2.resize(input_image, (512,512))

    # Setup
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)

    # Load obj file
    verts_rgb_colors = get_verts_rgb_colors(obj_path)
    verts_rgb_colors = torch.from_numpy(verts_rgb_colors).to(device)
    textures = TexturesVertex(verts_features=verts_rgb_colors)

    # Load obj
    mesh = load_objs_as_meshes([obj_path], device=device)

    # Set mesh
    vers = mesh._verts_list
    faces = mesh._faces_list
    mesh_w_tex = Meshes(vers, faces, textures)

    R, T = look_at_view_transform(1.8, 0, 0, device=device)
    images_w_tex = renderer(mesh_w_tex, R=R, T=T)
    images_w_tex = np.clip(images_w_tex[0, ..., :3].cpu().numpy(), 0.0, 1.0)[:, :, ::-1] * 255
    cv2.imwrite('./out1/'+filename, images_w_tex) 

    image = np.concatenate([input_image, images_w_tex], axis=1)
    cv2.imwrite('./out2/'+filename, image)


**３Dモデル画像生成**

In [ ]:
import os
import shutil
from tqdm import trange

if os.path.isdir('out1'):
     shutil.rmtree('out1')
os.makedirs('out1', exist_ok=True)

if os.path.isdir('out2'):
     shutil.rmtree('out2')
os.makedirs('out2', exist_ok=True)

for i in trange(num):
    filename = str(i).zfill(6)+'.png'
    out_img_path = '/content/pifuhd/results/pifuhd_final/recon/result_'+str(i).zfill(6)+'_256.png'
    obj_path = '/content/pifuhd/results/pifuhd_final/recon/result_'+str(i).zfill(6)+'_256.obj'
    renderer = set_renderer()
    generate_image_from_obj(obj_path, out_img_path, renderer)

## 画像から動画の生成

**３Dモデル**

In [ ]:
# 既に output1.mp4 があれば削除
import os
if os.path.exists('./output1.mp4'):
   os.remove('./output1.mp4')

! ffmpeg -r 10 -i out1/%6d.png\
               -vcodec libx264 -pix_fmt yuv420p output1.mp4

In [ ]:
# --- mp4動画の再生 ---
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output1.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="40%" height="40%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

**ビデオ＋3Dモデル**

In [ ]:
# 既に output2.mp4 があれば削除
import os
if os.path.exists('./output2.mp4'):
   os.remove('./output2.mp4')

! ffmpeg -r 10 -i out2/%6d.png\
               -vcodec libx264 -pix_fmt yuv420p output2.mp4

In [ ]:
# --- mp4動画の再生 ---
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output2.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="70%" height="70%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")